    # RoBERTa(A Robustly Optimized BERT Pretraining Approach)
- NSP(next sentence prediction) task 를 제거하고 MLM(Masked Language Model) task만으로 사전학습
- 더 많은 데이터와  더 오래 학습, 그리고 더 큰 mini-batch로 학습
- 동적 마스크 방식 사용(epoch 마자 중복 되지 않음)

# RoBERTa (A Robustly Optimized BERT Pretraining Approach)

## 1. 개요
- **RoBERTa**는 BERT의 개선 버전으로, BERT의 한계를 보완하고 성능을 최적화한 언어 모델.
- Facebook AI에 의해 개발되었으며, "더 긴 학습, 더 많은 데이터, 더 나은 하이퍼파라미터"를 강조함.
- 논문: *"RoBERTa: A Robustly Optimized BERT Pretraining Approach"*.

---

## 2. BERT와의 주요 차이점

| **특징**                  | **BERT**                                                                                       | **RoBERTa**                                                                                   |
|----------------------------|-----------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------|
| **사전학습 데이터**        | 16GB (BookCorpus + English Wikipedia)                                                         | 160GB (BookCorpus, English Wikipedia + Common Crawl News, OpenWebText, Stories)              |
| **MLM 마스크 방식**        | 사전 생성된 마스크 적용.                                                                      | 동적으로 생성된 마스크 사용: 각 배치마다 마스크 위치를 새로 샘플링.                           |
| **NSP (Next Sentence Prediction)** | 문장 연결 관계를 학습하기 위해 사용.                                                      | **제거**: 문장 연결 관계 학습이 모델 성능에 큰 기여를 하지 않는다고 판단.                    |
| **배치 크기**              | 상대적으로 작은 배치 크기 (256)                                                               | 큰 배치 크기 (8,000) 사용: 학습 안정성과 효율성 증가.                                         |
| **학습 반복 횟수**         | 1M 스텝                                                                                       | 125K~500K 스텝 동안 대규모 데이터셋에서 더 긴 학습 진행.                                      |
| **학습률 스케줄**          | Warmup + 선형 감소.                                                                           | BERT와 동일하지만 학습 반복 횟수와 배치 크기를 늘려 최적화.                                   |

---

## 3. 학습 방법의 구체적 변경점

### **3.1 동적 마스킹**
- BERT는 학습 전에 마스크된 데이터를 미리 생성.
- RoBERTa는 **동적 마스킹**을 도입:
  - 학습 중 각 배치마다 새로운 마스크 위치를 샘플링.
  - 동일한 문장이더라도 반복 학습 시 마스크 위치가 달라져 모델의 일반화 성능 향상.
  - ## RoBERTa의 동적 마스킹 (Dynamic Masking)

  **동적 마스킹**은 RoBERTa에서 사용되는 중요한 특징으로, 학습 데이터에서 **매번 배치마다 마스킹이 다르게 적용**되는 방식을 의미합니다. 이는 모델이 보다 다양한 문맥을 학습하도록 도와줍니다.

  ### 동적 마스킹 vs 고정 마스킹
  - **BERT**에서는 **고정된 마스킹** 방식을 사용합니다. 즉, 학습 데이터에서 **항상 같은 단어들**이 마스킹됩니다. 예를 들어, 입력 문장에서 일정 비율의 단어를 마스킹해놓고, 그 마스킹된 단어를 예측하는 방식입니다.
  
  - **RoBERTa**에서는 **동적 마스킹**을 사용하여, 배치가 학습될 때마다 **매번 다르게 마스킹**됩니다. 각 배치마다 **랜덤하게 마스킹**이 적용되므로, 모델은 다양한 문맥을 학습할 수 있습니다.

  ### 동적 마스킹의 장점
  1. **다양성 증가**: 배치마다 마스킹 위치가 달라지기 때문에, 모델은 다양한 문맥을 학습할 수 있습니다.
  2. **과적합 방지**: 고정된 마스킹 방식에서 발생할 수 있는 과적합을 방지할 수 있습니다.
  3. **일반화 성능 향상**: 더 많은 패턴을 학습하면서 모델은 더 잘 일반화할 수 있습니다.

  ### 예시
  - **BERT의 고정 마스킹**: 예를 들어, 문장이 `The cat sat on the mat`일 때, 고정 마스킹 방식에서는 항상 `[MASK]`가 같은 위치에 적용됩니다. 예를 들어, `The [MASK] sat on the mat`처럼 특정 위치의 단어가 항상 마스킹됩니다.
  
  - **RoBERTa의 동적 마스킹**: 동일한 문장에서 RoBERTa는 매 배치마다 마스킹된 위치가 달라집니다. 예를 들어, 첫 번째 배치에서는 `The cat [MASK] on the mat`, 두 번째 배치에서는 `The [MASK] sat on the mat`와 같이 마스킹된 위치가 달라집니다.

  ### 결론
  - **동적 마스킹**은 배치마다 마스킹 위치가 달라지는 방식으로, 이를 통해 RoBERTa 모델은 더 다양한 문맥을 학습할 수 있습니다.
  - RoBERTa는 BERT에서의 고정된 마스킹 방식 대신 동적 마스킹을 사용하여 성능을 개선하였습니다.

### **3.2 NSP 제거**
- BERT는 **Next Sentence Prediction (NSP)**를 사용해 문장 간 논리적 연결성을 학습.
- RoBERTa는 NSP 태스크를 완전히 제거:
  - 이유: NSP가 MLM 학습 성능에 큰 영향을 주지 않으며, 불필요한 복잡성을 초래.
  - 학습 대신, 긴 시퀀스(512 토큰)로 텍스트를 구성하여 문맥을 학습.

### **3.3 대규모 데이터셋 사용**
- BERT가 사용한 16GB 데이터보다 **10배 이상 많은 데이터**를 활용:
  - BookCorpus, Wikipedia 외에 **Common Crawl News, OpenWebText, Stories** 추가.
  - 데이터를 다양화해 더 일반적인 언어 이해 능력을 학습.

### **3.4 큰 배치 크기와 긴 학습**
- RoBERTa는 배치 크기를 **8,000**으로 크게 늘림.
- 학습 반복 횟수는 BERT보다 훨씬 길게 설정하여 더 많은 텍스트를 학습.

---

## 4. RoBERTa의 구체적 학습 예시

### **입력 데이터**
```plaintext
Original Sentence: "Machine learning is fascinating and rapidly evolving."
Masked Input: "[MASK] learning is fascinating and [MASK] evolving."
```

## 훈련 과정
Masked Language Modeling (MLM):

[MASK]를 원래 단어로 복원하도록 학습.
모델이 "Machine"과 "rapidly"를 예측.

## 동적 마스킹:

같은 문장이지만, 다음 배치에서 마스크 위치가 변경.
```python
예:

배치 1: "[MASK] learning is fascinating and [MASK] evolving."
배치 2: "Machine [MASK] is [MASK] and rapidly evolving."
```
## NSP 제거:

문장 간 관계를 학습하지 않고 긴 문맥 내에서 단어 간 관계만 학습.
최종 결과:

모델이 문맥에 따라 "Machine"과 "rapidly"를 성공적으로 복원.

## 5. 결과
RoBERTa는 여러 벤치마크에서 BERT를 능가:
GLUE, SQuAD, RACE 등 다양한 NLP 태스크에서 높은 성능.
RoBERTa의 설계 철학: 기존 아키텍처(BERT)를 최적화해 더 강력한 결과를 도출.

## 6. 요약
RoBERTa는 BERT의 구조를 유지하면서 데이터, 하이퍼파라미터, 학습 방식을 개선해 성능을 극대화.
동적 마스킹, NSP 제거, 대규모 데이터, 긴 학습 등이 주요 특징.
결과적으로 더 강력하고 일반화된 언어 모델을 제공.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = os.getcwd()+'/data/review/'
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
train = pd.read_csv(f"{DATA_PATH}review_train.csv")
test = pd.read_csv(f"{DATA_PATH}review_test.csv")
train.shape, test.shape

((2000, 3), (1000, 2))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2000 non-null   object
 1   review  2000 non-null   object
 2   target  2000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.0+ KB


In [5]:
train.head()

,id,review,target
0,train_0,이런 최고의 영화를 이제서야 보다니,1
1,train_1,안봤지만 유승준나와서 비추.,0
2,train_2,시대를 못 따라간 연출과 촌스러운 영상미.,0
3,train_3,원소전 굿,1
4,train_4,ㅋㅋㅋㅋ 개봉영화평점단사람이1명 ㅋㅋㅋㅋ,1


In [6]:
model_name = "ehdwns1516/klue-roberta-base-kornli"

In [7]:
from transformers import AutoTokenizer, AutoModel

In [8]:
model = AutoModel.from_pretrained(model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at ehdwns1516/klue-roberta-base-kornli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
tokenizer.model_max_length

512

In [12]:
text = train["review"][0]
text

'이런 최고의 영화를 이제서야 보다니'

In [13]:
tokenizer(text, padding="max_length", truncation=True)

{'input_ids': [0, 3667, 3841, 2079, 3771, 2138, 3699, 15469, 3632, 2209, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# 학습데이터 -> ndarray

In [14]:
train_arr = train["review"].to_numpy()
test_arr = test["review"].to_numpy()

# 정답데이터

In [15]:
target = train["target"].to_numpy().reshape(-1,1)

# 데이터셋 클래스

In [16]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, x, y=None):
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.get_tokenizer(self.x[idx])
        
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item
    def get_tokenizer(self, text):
        x = self.tokenizer(text, padding="max_length", truncation=True)
        for k, v in x.items():
            x[k] = torch.tensor(v)
        return x

In [17]:
dt = ReviewDataset(tokenizer, train_arr, target)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[   0, 3667, 3841,  ...,    1,    1,    1],
         [   0, 1378, 3072,  ...,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[1.],
         [0.]])}

In [18]:
outputs = model(**batch["x"])

In [19]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [20]:
outputs["last_hidden_state"].shape # 모든 토큰의 출력

torch.Size([2, 512, 768])

In [21]:
outputs["pooler_output"].shape # CLS 토큰을 FC 레이어에 통과 시켜서 나온 출력

torch.Size([2, 768])

In [22]:
outputs[0].shape

torch.Size([2, 512, 768])

In [23]:
outputs[1].shape

torch.Size([2, 768])

In [24]:
model.config.hidden_size # d_model 사이즈

768

In [25]:
model.config.max_position_embeddings # 최대 시퀀스 사이즈

512

In [33]:
def test_unpacking(**kwargs):
    # 언팩된 인자들을 출력
    for key, value in kwargs.items():
        print(f"{key}: {value}")


test_unpacking(**batch)

x: {'input_ids': tensor([[   0, 3667, 3841,  ...,    1,    1,    1],
        [   0, 1378, 3072,  ...,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
y: tensor([[1.],
        [0.]])


# 모델 클래스

In [26]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pre_model = AutoModel.from_pretrained(model_name)
        self.fc_out = torch.nn.Linear( self.pre_model.config.hidden_size, 1)

    def forward(self, x):
        x = self.pre_model(**x)
        # x[0]: 모든 시점의 히든출력 batch, seq, features
        # x[1]: CLS 토큰의 히든출력 batch, features
        return self.fc_out(x[1])

In [27]:
model = Net(model_name)
model(batch["x"])

Some weights of RobertaModel were not initialized from the model checkpoint at ehdwns1516/klue-roberta-base-kornli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tensor([[-0.2606],
        [-0.1564]], grad_fn=<AddmmBackward0>)

# 학습 loop 함수

In [32]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in dataloader:
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

# 검증및 테스트 loop 함수

In [31]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in dataloader:
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn( pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred

# 하이퍼파라미터

In [30]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=SEED)

batch_size = 16 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

model_name = 'ehdwns1516/klue-roberta-base-kornli' # 사전학습 모델 id
tokenizer = AutoTokenizer.from_pretrained(model_name) # 지정한 사전학습 모델의 토크나이저

# 학습

In [35]:
is_holdout = False
reset_seeds(SEED) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_arr) ):
    # 학습용 데이터로더 객체
    train_dt = ReviewDataset(tokenizer, train_arr[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = ReviewDataset(tokenizer, train_arr[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = (pred > 0.5).astype(int) # 이진분류 문제에서 클래스 번호 결정
        score = accuracy_score(target[vai], pred)

        #print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"{DATA_PATH}/weight/roberta_model_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 정확도: {best_score}")
    best_score_list.append(best_score)

    del model # 변수 삭제
    import gc
    gc.collect() # 메모리 청소
    torch.cuda.empty_cache()  # gpu 메모리 청소

    if is_holdout:
        break

Some weights of RobertaModel were not initialized from the model checkpoint at ehdwns1516/klue-roberta-base-kornli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/100 [00:00<?, ?it/s]

0번째 폴드 최고 정확도: 0.8425


Some weights of RobertaModel were not initialized from the model checkpoint at ehdwns1516/klue-roberta-base-kornli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

3번째 폴드 최고 정확도: 0.865


Some weights of RobertaModel were not initialized from the model checkpoint at ehdwns1516/klue-roberta-base-kornli and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/100 [00:00<?, ?it/s]

4번째 폴드 최고 정확도: 0.855
